In [ ]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 10.0 MB/s eta 0:00:00


In [3]:
#import optuna
import torch
from torch import optim, nn
import torch.nn.functional as F
from Model import resnet18_model_fine_tuning, resnet18_model_freezed
import DataLoader

ModuleNotFoundError: ignored

In [ ]:
# device - cpu or gpu?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size = 64
epochs = 10
log_interval = 10
n_train_examples = batch_size * 30
n_valid_examples = batch_size * 10

def get_model():
  return resnet18_model_fine_tuning

def objective(trial):
  # Generate the model.
  model = get_model() # Already on device
  
  # Generate the optimizers.
  lr = trial.suggest_float("lr", 1e-7, 1e-4, log=True) # log=True, will use log scale to interplolate between lr
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  
  criterion = nn.CrossEntropyLoss()

  # Get Data loader
  train_dataset = DataLoader.VoxCaleb1MelSpecDataset('./drive/MyDrive/VoxCeleb1', 'train', input_size=224, download=False)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2)
  valid_dataset = DataLoader.VoxCaleb1MelSpecDataset('./drive/MyDrive/VoxCeleb1', 'val', input_size=224, download=False)
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
  # Training of the model.
  for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      # Limiting training data for faster epochs.
      if batch_idx * batch_size >= n_train_examples:
        break

      data, target = data.to(device), target.to(device)
      
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
    
    # Validation of the model.
    model.eval()
    correct = 0
    with torch.no_grad():
      for batch_idx, (data, target) in enumerate(valid_loader):
        # Limiting validation data.
        if batch_idx * batch_size >= n_valid_examples:
          break
        data, target = data.to(device), target.to(device)
        output = model(data)
        # Get the index of the max log-probability.
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
      
    accuracy = correct / min(len(valid_loader.dataset), n_valid_examples)
    
    # report back to Optuna how far it is (epoch-wise) into the trial and how well it is doing (accuracy)
    trial.report(accuracy, epoch)
    # then, Optuna can decide if the trial should be pruned
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()
      
  return accuracy


In [23]:
# now we can run the experiment
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name="resnet-fc", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print("Study statistics: ")
print(" Number of finished trials: ", len(study.trials))
print(" Number of pruned trials: ", len(pruned_trials))
print(" Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print(" Value: ", trial.value)
print(" Params: ")

for key, value in trial.params.items():
 print(" {}: {}".format(key, value))

[I 2023-01-15 15:42:11,835] A new study created in memory with name: resnet-fc
[I 2023-01-15 17:10:48,941] Trial 0 finished with value: 0.0328125 and parameters: {'lr': 1.9990262624334293e-06, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.0328125.
[I 2023-01-15 18:39:35,425] Trial 1 finished with value: 0.0421875 and parameters: {'lr': 5.477832478778276e-07, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.0421875.
[I 2023-01-15 20:07:49,938] Trial 2 finished with value: 0.0421875 and parameters: {'lr': 2.1499185506784375e-06, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.0421875.
[I 2023-01-15 21:37:32,996] Trial 3 finished with value: 0.0421875 and parameters: {'lr': 3.846162088975758e-07, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.0421875.
[I 2023-01-15 23:07:15,119] Trial 4 finished with value: 0.0421875 and parameters: {'lr': 2.6204922821359317e-06, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.0421875.
[I 2023-01-16 00:37:12,734] Trial 5 finished wit

Study statistics: 
 Number of finished trials:  20
 Number of pruned trials:  11
 Number of complete trials:  9
Best trial:
 Value:  0.046875
 Params: 
 lr: 1.0194986974126253e-06
 optimizer: Adam


In [1]:
optuna.visualization.plot_param_importances(study)

NameError: ignored

In [ ]:
optuna.visualization.plot_contour(study, params=["n_units_l0", "dropout_l0"])
